In [0]:
dbutils.library.restartPython()

# Import de Funções e Libs

As bibliotecas e funções utilizadas ao longo do código devem ser adicionadas aqui

In [0]:
import numpy as np
import pandas as pd
import requests
import json
import sherlock
import regras_politica as rp

### Testes Modulares - Referente ao tópico - Definição de Grupos Antigos

##### Geração de Massa

In [0]:
#Gerando a massa modelo passando o caminho do JSON como nome da variável
base_entrada = pd.DataFrame({'payloadHomol.intermediarias.clienteFoPag':[0],
                             'payloadHomol.intermediarias.principalidadeDeInvestimento':[0],
                             'solicitante.flagCartaoDebitoContratado':[0],
                             'payloadHomol.intermediarias.flagClienteExistente':[0],
                             'solicitante.saldoMinimo30d':[99]
                            })

In [0]:
#Criação das massas com casos forçados
massa_definicaoGruposAntigos = sherlock.Gera_Massa(base_entrada)
massa_definicaoGruposAntigos.gera_cenario('payloadHomol.intermediarias.clienteFoPag',[0,1])
massa_definicaoGruposAntigos.gera_cenario('payloadHomol.intermediarias.principalidadeDeInvestimento',[0,1])
massa_definicaoGruposAntigos.gera_cenario('solicitante.flagCartaoDebitoContratado',[0,1])
massa_definicaoGruposAntigos.gera_cenario('payloadHomol.intermediarias.flagClienteExistente',[0,1])
massa_definicaoGruposAntigos.gera_cenario('solicitante.saldoMinimo30d',[-1,0,99,100,101])

massa_definicaoGruposAntigos.get_massa_final()

massa_definicaoGruposAntigos.set_bom_arqv()
massa_definicaoGruposAntigos.converte_massa()

### Execução Modular e Geração de CSV

In [0]:
listaResultado = rp.execucaoModular(massa_definicaoGruposAntigos.get_massa_json(),rp.DefinicaoGruposAntigo)

##### Valida se todas condições estão sendo abordadas

In [0]:
import pandas as pd
results = []
for caso in listaResultado:
    results.append(caso['payloadHomol']['intermediarias']['NovoGrupo'])
    
freqzinha = pd.DataFrame(results, columns=['valor'])
freqzinha['Freq'] = freqzinha['valor'].map(freqzinha['valor'].value_counts())
freqzinha = freqzinha.drop_duplicates().reset_index(drop=True)
freqzinha

In [0]:
# Renomeação das colunas para o cabeçalho do RMA
massa_definicaoGruposAntigos.rename_cabecalho()

In [0]:
massa_definicaoGruposAntigos.get_massa_dm()

In [0]:
massa_definicaoGruposAntigos.atribui_expected(listaResultado,"payloadHomol.intermediarias.NovoGrupo","NovoGrupo")
massa_definicaoGruposAntigos.get_massa_dm()

##### Conversão da massa para CSV

In [0]:
massa_definicaoGruposAntigos.get_massa_dm().to_csv(r'/Workspace/Users/kaue.ongaro@picpaybank.com/2 - Homologação/Concessao de Cartoes - Online/massas/massaDefinicaoGruposAntigos.csv',index=False)